# Analysing M&A Transactions

This Jupyter Notebook contains analysis of the efficiency of the M&A transactions. Three research questions are asked:

1. Is it the case that M\&A improves the efficiency of merging companies?
2. Which of the two entities benefits the most?  
3. Can Decision Makers utilise synergies to increase efficiency?

In [1]:
import numpy as np
import pandas as pd
import xpress as xp
from DEAmodels import *
from scipy.stats import mannwhitneyu

xp.setOutputEnabled(False)

The license file was located using the XPRESSDIR environment variable: C:\xpressmp\bin\xpauth.xpr. To use an alternative license file, set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr. NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.



InterfaceError: Xpress licensing error 14: Failed to connect to the license server on mathsvm1.maths.ed.ac.uk (129.215.255.69) for reason: Connection to target server timed out

# 1. Is it the case that M&A improves the efficiency of merging companies

In [232]:
DMUnum = 45*2

The data is loaded and a list of names is created that will be passed to the DEA function. The file NewnamesBank.xlsx are the changed names of the banks. This is needed as the original names would result in an error since there are duplicates.

In [233]:
InputMatrix = np.loadtxt("Data/DSBMInputMatrix.txt").reshape(2,DMUnum,5)
OutputMatrix = np.loadtxt("Data/DSBMOutputMatrix.txt").reshape(2,DMUnum,5)
CarryOverMatrix = np.loadtxt("Data/DSBMCarryOverMatrix.txt").reshape(1,DMUnum,5)
MAnames = pd.read_excel("Data/NewNamesBank.xlsx",header = None).rename(columns ={0:"ACQ_ID",1:"ACQ_Name",2:"TAR_ID",3:"TAR_Name"})

In [234]:
names = []
for i in range(MAnames.shape[0]):
    IDACQ = MAnames["ACQ_Name"].to_numpy()[i]
    IDTAR = MAnames["TAR_Name"].to_numpy()[i]

    names.append(IDACQ)
    names.append(IDTAR)

For each of the possible orientations the results are created:

In [235]:
resultsNon = DSBMModel(input = InputMatrix,output = OutputMatrix, carryfix = CarryOverMatrix,rts = "VRS",return_type=1,orientation="Non-Oriented",names = names)
resultsInput = DSBMModel(input = InputMatrix,output = OutputMatrix, carryfix= CarryOverMatrix,rts = "VRS",return_type=1,orientation="Input",names = names)
resultsOutput = DSBMModel(input = InputMatrix,output = OutputMatrix, carryfix = CarryOverMatrix,rts = "VRS",return_type=1,orientation="Output",names = names)

The following are the results for the Non-Oriented model.

In [236]:
pd.set_option('display.max_rows', None)
resultsNon.head(90)

,TotalScore,PeriodScoreMin,PeriodScoreMax,PeriodScoreAvg,AbsGap
"TD Bank, NA",1.000000,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"
Scottrade Bank,1.000000,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"
Manufacturers & Traders Trust Co,1.000000,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"
Hudson City Savings Bank,1.000000,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"
"CIT Bank, NA",1.000000,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"
CIT Bank,1.000000,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"
Frost Bank,0.877095,"[0.6682854478421189, 0.8300506043709381, 1.0, ...","[0.6682854478421194, 0.8300506043709386, 1.000...","[0.6682854478421192, 0.8300506043709384, 1.000...","[5.551115123125783e-16, 4.440892098500626e-16,..."
Western National Bank,0.894666,"[0.6953785589569275, 0.8675553779362108, 0.999...","[0.6953785589569285, 0.8675553779362155, 1.000...","[0.6953785589569279, 0.8675553779362132, 0.999...","[9.992007221626409e-16, 4.6629367034256575e-15..."
HanCock Bank,1.000000,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"
Whitney Bank,1.000000,"[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[1.0, 1.0, 1.0, 1.0, 1.0]","[0.0, 0.0, 0.0, 0.0, 0.0]"


To test whether an efficiency increase has taken place pre and post merger efficiencies are computed according to formula (5.1) and (5.2) from the Dissertation.

In [237]:
resAvgNon = resultsNon["PeriodScoreAvg"].to_list()
resAvgInput = resultsInput["PeriodScoreAvg"].to_list()
resAvgOutput = resultsOutput["PeriodScoreAvg"].to_list()

In [238]:
resAvgNon = np.array(resAvgNon)
resAvgInput = np.array(resAvgInput)
resAvgOutput = np.array(resAvgOutput)

The pre and post merger efficiencies are created:

In [239]:
preNon = resAvgNon[:,0:2].mean(axis = 1)
postNon = resAvgNon[:,3:].mean(axis = 1)
overallNon = resultsNon["TotalScore"]

preInput = resAvgInput[:,0:2].mean(axis = 1)
postInput = resAvgInput[:,3:].mean(axis = 1)
overallInput = resultsInput["TotalScore"]

preOutput = resAvgOutput[:,0:2].mean(axis = 1)
postOutput = resAvgOutput[:,3:].mean(axis = 1)
overallOutput = resultsOutput["TotalScore"]

Finally the Mann-Whitney-U test is used to test differences between the pre and post efficiency.

In [240]:
print(mannwhitneyu(preNon,postNon))
print(mannwhitneyu(preInput,postInput))
print(mannwhitneyu(preOutput,postOutput))

MannwhitneyuResult(statistic=2787.5, pvalue=0.00028892050065893983)
MannwhitneyuResult(statistic=2808.0, pvalue=0.0003357724446494892)
MannwhitneyuResult(statistic=2539.0, pvalue=1.1047163664049294e-05)


## 2. Which of the two entities benefits the most?  

To compare which kind of entity benefits the most the differences are calculated according to formula (5.3) from the Dissertation.

In [242]:
DiffNon = postNon - preNon
DiffInput = postInput - preInput
DiffOutput = postOutput - preOutput

The differences are split into Targets and Acquirers.

In [243]:
DiffNonACQ = []
DiffNonTAR = []
DiffInputACQ = []
DiffInputTAR = []
DiffOutputACQ = []
DiffOutputTAR = []

j = 0
for i in range(45):
    DiffNonACQ.append(DiffNon[j])
    DiffInputACQ.append(DiffInput[j])
    DiffOutputACQ.append(DiffOutput[j])

    DiffNonTAR.append(DiffNon[j+1])
    DiffInputTAR.append(DiffInput[j+1])
    DiffOutputTAR.append(DiffOutput[j+1])

    j+=2


Finally the Mann-Whitney-U test is used to test differences between the two groups.

In [244]:
print(mannwhitneyu(DiffNonACQ,DiffNonTAR))
print(mannwhitneyu(DiffInputACQ,DiffInputTAR))
print(mannwhitneyu(DiffOutputACQ,DiffOutputTAR))

MannwhitneyuResult(statistic=1068.5, pvalue=0.653339964999999)
MannwhitneyuResult(statistic=1060.0, pvalue=0.7036916538360445)
MannwhitneyuResult(statistic=1095.0, pvalue=0.5067229370467121)


## 3. Can Decision Makers utilise synergies to increase efficiency?

To analyse efficiency gains from synergy the SBM model is solved for both actual and virtual banks

In [246]:
InputMatrix = np.loadtxt("Data/SBMInputMatrix.txt").reshape(3,DMUnum)
OutputMatrix = np.loadtxt("Data/SBMOutputMatrix.txt").reshape(2,DMUnum)

MAnames = pd.read_excel("Data/NewNamesBank.xlsx",header = None).rename(columns ={0:"ACQ_ID",1:"ACQ_Name",2:"TAR_ID",3:"TAR_Name"})

In [247]:
names = []

for i in range(MAnames.shape[0]):
    curname = MAnames['ACQ_Name'][i]

    names.append(f"{curname}")
    # names.append(f"{curname} Virtual")

The SBM Model is used to obtain the efficiency scores

In [248]:
resultsNon = SBMModel(input = InputMatrix, output= OutputMatrix, rts = "VRS",return_type=1,orientation="Non-Oriented")
resultsInput = SBMModel(input = InputMatrix, output= OutputMatrix, rts = "VRS",return_type=1,orientation="Input")
resultsOutput = SBMModel(input = InputMatrix, output= OutputMatrix, rts = "VRS",return_type=1,orientation="Output")

The results are split into Actual and Virtual DMUs

In [249]:
EffNonAct = []
EffNonVirt = []
EffInputAct = []
EffInputVirt = []
EffOutputAct = []
EffOutputVirt = []

j = 0
for i in range(45):
    EffNonAct.append(resultsNon["Score"][j])
    EffInputAct.append(resultsInput["Score"][j])
    EffOutputAct.append(resultsOutput["Score"][j])

    EffNonVirt.append(resultsNon["Score"][j+1])
    EffInputVirt.append(resultsInput["Score"][j+1])
    EffOutputVirt.append(resultsOutput["Score"][j+1])

    j+=2


Finally the Mann-Whitney-U test is used to test for differences between virtual and actual banks.

In [252]:
print(mannwhitneyu(EffNonAct,EffNonVirt))
print(mannwhitneyu(EffInputAct,EffInputVirt))
print(mannwhitneyu(EffOutputAct,EffOutputVirt))

MannwhitneyuResult(statistic=1022.0, pvalue=0.9408804581761264)
MannwhitneyuResult(statistic=1004.0, pvalue=0.9470830940579014)
MannwhitneyuResult(statistic=1018.5, pvalue=0.9639202093337544)
